Fashion MNIST 분류 예제를 만들어봅시다.

Fully Connected Layer로 모델을 만들어 봅시다.

https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

pytorch에서 제공하는 데이터셋을 활용해서 진행해보겠습니다.

1. 라이브러리 불러오기

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

2. 데이터 불러오기

- DataLoader를 통해서 데이터를 불러옵니다
- Train과 Test 데이터셋을 분리해서 가져옵니다.

https://pytorch.org/vision/main/generated/torchvision.datasets.FashionMNIST.html

In [2]:
# Train dataset
training_data = datasets.FashionMNIST(
    root="data",    # 저장할 위치
    train=True,     # 훈련 데이터셋
    download=True,  # 다운로드 실행
    transform=ToTensor(),   # 변환할 함수
)

# Test Dataset
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:02<00:00, 11163859.15it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 201993.72it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3749111.89it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 3486561.76it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



파일 창을 살펴보면 data 폴더가 생성되고, 데이터가 저장된 것을 확인할 수 있습니다.


데이터셋을 살펴봅시다

pytorch 데이터셋은 DataLoader를 사용하여 batch단위로 학습할 수 있습니다.

In [3]:
batch_size = 64

# 데이터 로더 만들기
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


모델을 만들어 봅시다

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()  # 이미지를 1차원으로 변경
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x

In [5]:
model = NeuralNetwork()

gpu 사용을 해봅시다.


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"  # gpu 사용 가능 여부 확인, 없으면 cpu로 사용
print(device)

cuda


In [7]:
model = model.to(device)

In [8]:
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

모델의 파라미터 수 등을 살펴봅시다.
torchsummary 라이브러리를 사용해보겠습니다

In [9]:
from torchsummary import summary
summary(model,input_size = (1,28,28) )   # 총 66만개가 넘는 파라미터가 생성되었습니다.

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 512]         401,920
              ReLU-3                  [-1, 512]               0
            Linear-4                  [-1, 512]         262,656
              ReLU-5                  [-1, 512]               0
            Linear-6                   [-1, 10]           5,130
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.55
Estimated Total Size (MB): 2.58
----------------------------------------------------------------


loss fuction과 optimaze 지정


In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

재사용을 위해 학습코드를 함수로 만듭니다

In [11]:
def train(dataloader, model, loss_fn, optimizer):  # 모델이나, loss_fn 등을 바꿔도 사용할 수 있도록 입력값으로 처리리
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader): # enumerate은 인덱스 숫자 추가가
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

테스트 코드도 만들어 보겠습니다.

이때는 레이어의 학습이 불필요하기 때문에 약간의 수정이 필요합니다

* model.eval(): 추론 모드로 변경하여 Dropout, BatchNorm 등 레이어에 랜덤요소 있는 부분 영향 없도록 함
* torch.no_grad(): 그래디언트 계산 안 함, 성능 향상

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

훈련과 평가를 동시에 해봅시다

In [13]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.322296  [   64/60000]
loss: 2.307023  [ 6464/60000]
loss: 2.287705  [12864/60000]
loss: 2.269243  [19264/60000]
loss: 2.262988  [25664/60000]
loss: 2.225229  [32064/60000]
loss: 2.232006  [38464/60000]
loss: 2.201263  [44864/60000]
loss: 2.204703  [51264/60000]
loss: 2.160538  [57664/60000]
Test Error: 
 Accuracy: 36.1%, Avg loss: 2.162716 

Epoch 2
-------------------------------
loss: 2.186265  [   64/60000]
loss: 2.171175  [ 6464/60000]
loss: 2.113581  [12864/60000]
loss: 2.116169  [19264/60000]
loss: 2.078167  [25664/60000]
loss: 2.012525  [32064/60000]
loss: 2.037413  [38464/60000]
loss: 1.960934  [44864/60000]
loss: 1.977279  [51264/60000]
loss: 1.891310  [57664/60000]
Test Error: 
 Accuracy: 52.9%, Avg loss: 1.895088 

Epoch 3
-------------------------------
loss: 1.942944  [   64/60000]
loss: 1.901891  [ 6464/60000]
loss: 1.787796  [12864/60000]
loss: 1.813451  [19264/60000]
loss: 1.715700  [25664/60000]
loss: 1.671360  [32064/600